In [17]:
import pandas as pd
import numpy as np
from dfply import *


In [113]:
snps_data = pd.read_csv(
    '/home/memon/genetic_analyses/ann/classifier/snps_Send_Asif/Asif_Genotype_Disease_Only_ROSMAP.CSV', index_col=0
)
snps_data = snps_data.transpose()

In [114]:
snps_data.head()

SNP,rs7539455,rs6687381,rs4846028,rs12095517,rs4073574,rs4845875,rs4845877,rs4845881,rs4846048,rs6541003,...,rs7815138,rs1877829,rs2221222,rs6471533,rs7011044,rs2318878,rs2318876,rs3104965,rs2873824,rs4524756
MAP01797756,0,1,0,1,0,1,1,1,1,1,...,1,1,0,0,0,1,0,0,2,0
MAP05522533,1,0,0,1,0,1,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1
MAP07265221,1,0,1,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,0,0,1
MAP21362537,0,1,1,0,0,0,0,0,0,1,...,2,1,0,0,1,0,0,0,0,2
MAP24073245,1,0,0,1,1,2,2,1,1,2,...,2,2,0,2,0,1,0,0,0,0


In [115]:
snps_list = pd.read_csv(
    '/home/memon/genetic_analyses/approach_5/subgraphs15_snps_mod1.csv',header=None, index_col=0
)

subgraph2snps = {
    index: snps.dropna().tolist()
    for index, snps in snps_list.iterrows()
}

In [117]:
snp_mat = [
    snps_data[snps]
    for subgraph, snps in subgraph2snps.items()
]

In [121]:
snp_mat

[SNP          rs991108  rs7220419  rs2001381  rs417968  rs1635291  rs7224296  \
 MAP01797756         0          0          1         0          0          0   
 MAP05522533         0          0          2         1          1          1   
 MAP07265221         0          0          0         0          0          0   
 MAP21362537         0          0          2         0          0          0   
 MAP24073245         0          0          2         0          0          0   
 ...               ...        ...        ...       ...        ...        ...   
 ROS20501668         0          0          0         0          0          0   
 ROS20594407         1          1          0         1          1          1   
 ROS20674902         0          0          0         0          0          0   
 ROS21000928         0          0          0         1          1          1   
 ROS21001357         0          0          0         1          1          1   
 
 SNP          rs415430  rs378140  rs43